In [1]:
from jax_circuits import*
from topology import *
from penalty import *
import matplotlib.pyplot as plt

from qiskit import transpile
from qiskit.quantum_info import Operator

In [2]:
class TestCircuit():
    def __init__(self, ansatz, angles=None, circuit=None):
        self.ansatz = ansatz
        self.angles = angles
        if self.angles is not None:
            self.circuit = self.ansatz.circuit(self.angles)
        else:
            self.circuit = circuit

# Circuits with random angles

## 3q

Fully connected topology.

In [3]:
num_qubits = 3
gate_counts = list(range(15))[::3]
key = random.PRNGKey(0)

circuits_3conn = []
for n_gates in gate_counts:
    anz = Ansatz(num_qubits, 'cz', fill_layers(sequ_layer(num_qubits), n_gates))
    key, subkey = random.split(key)
    angles = random_angles(anz.num_angles, key=subkey)
    tc = TestCircuit(anz, angles)
    circuits_3conn.append(tc)

Chain topology.

In [4]:
num_qubits = 3
gate_counts = list(range(15))[::3]
key = random.PRNGKey(0)

circuits_3chain = []
for n_gates in gate_counts:
    anz = Ansatz(3, 'cz', fill_layers(chain_layer(3), n_gates))
    key, subkey = random.split(key)
    angles = random_angles(anz.num_angles, key=subkey)
    tc = TestCircuit(anz, angles)
    circuits_3conn.append(tc)

## 4q

Fully connected topology.

In [5]:
num_qubits = 4
gate_counts = list(range(64))[::10]
key = random.PRNGKey(0)

circuits_4conn = []
for n_gates in gate_counts:
    anz = Ansatz(num_qubits, 'cz', fill_layers(sequ_layer(num_qubits), n_gates))
    key, subkey = random.split(key)
    angles = random_angles(anz.num_angles, key=subkey)
    tc = TestCircuit(anz, angles)
    circuits_4conn.append(tc)

Chain topology.

In [6]:
num_qubits = 4
gate_counts = list(range(64))[::10]
key = random.PRNGKey(0)

circuits_4chain = []
for n_gates in gate_counts:
    anz = Ansatz(num_qubits, 'cz', fill_layers(chain_layer(num_qubits), n_gates))
    key, subkey = random.split(key)
    angles = random_angles(anz.num_angles, key=subkey)
    tc = TestCircuit(anz, angles)
    circuits_4chain.append(tc)

# Toffoli gates

## 3q

In [7]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
u_toff3 = Operator(qc.reverse_bits()).data
qc_toff3_trans = transpile(qc, basis_gates=['cz', 'rx', 'ry', 'rz'])

placements_toff3 = [[1, 2], [0, 2], [1, 2], [0, 2], [0, 1], [0, 1]]
anz_toff3 = Ansatz(3, 'cz', placements={'free': placements_toff3})
tc_toff3 = TestCircuit(anz_toff3, circuit=qc_toff3_trans)

## 4q

Fully connected.

In [8]:
qc = QuantumCircuit(4)
qc.mct([0, 1, 2], 3)
u_toff4 = Operator(qc.reverse_bits()).data
qc_toff4_trans = transpile(qc, basis_gates=['cz', 'rx', 'ry', 'rz'])

placements_toff4 = [[0, 1], [0, 1], [1, 2], [0, 2], [1, 2], [0, 2], [2, 3], [1, 3], [2, 3], [0, 3], [2, 3], [1, 3], [2, 3], [0, 3]]
anz_toff4 = Ansatz(4, 'cz', placements={'free': placements_toff4})
tc_toff4 = TestCircuit(anz_toff4, circuit=qc_toff4_trans)

Chain. Optimization level=3 probably uses stochastic swaps and the result is run-dependent. The best I've got is 21 gates for chain topology. Note that this probably does not include swapping the qubits back in order.

In [11]:
qc = QuantumCircuit(4)
qc.mct([0, 1, 2], 3)
u_toff4 = Operator(qc.reverse_bits()).data
qc_toff4_trans_chain = transpile(qc, basis_gates=['cz', 'rx', 'ry', 'rz'],
                                 coupling_map=[[0,1], [1,0], [1,2],[2,1],[2,3],[3,2]], optimization_level=3)

(qc_toff4_trans_chain.qasm()).count('\ncz') 

23

T-shaped [ref](http://arxiv.org/abs/2109.13223)

In [10]:
toff4_t_placements = [[2, 3], [1, 3], [0, 3], [1, 3], [0, 3], [2, 3], [0, 3], [1, 3], [2, 3], [0, 3], [2, 3], [0, 3], [1, 3], [2, 3], [0, 3], [2, 3], [1, 3]]
# toff4_t_1q_gates = (too lazy to write)